# Clustering Zooniverse Marks to count Iguanas
The goal is to find the best method to cluster the data and find the best number of clusters.
The benchmark is a gold standard dataset obtained by experts.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
sys.path.append("../zooniverse")

## Intro
### Retrieve a Classification report from Zooniverse
Export the classification export from your zooniverse project.
https://www.zooniverse.org/lab/11905/data-exports

This leads to a csv file which can be used for the analysis which should be renamed to `iguanas-from-above-classifications.csv` and placed in the `input_path` directory.
The methods do not use methods from zooniverse. It is a custom implementation.

An alternative would be to use the [code provided by zooniverse](https://github.com/zooniverse/Data-digging/tree/master/notebooks_ProcessExports)
(Bird Count Example)[https://github.com/zooniverse/Data-digging/blob/master/scripts_ProjectExamples/seabirdwatch/bird_count.py]

This notebooks assumes the data is flat and prepared. An alternative format would be the [cesar aggregation format](https://github.com/zooniverse/aggregation-for-caesar)

Used Methods are:

### DBSCAN 
It does not require the number of clusters to be specified. It is used here because, but has min_samples and eps as hyperparameters which need to be found. [Link](https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html)
For finding eps and min_sample a simple **grid search** is used.
Additionally, DBSCAN not assume a specific shape for the clusters (K-means assumes clusters are gaussian in shape) even though we should assume that points around an iguana is gaussian shaped.

### HDBSCAN
It is an extension of DBSCAN which is more robust to hyperparameter settings as it finds epsilon and min_samples automatically. [Link](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html)

### Processing workflow

The Data is flatted and filtered
![Image](images/dataprocessing-DataFiltering.png)

In each phase we have the following number of images if they are filtered for at least 4 true marks by users
1. phase 105
2. phase 160
3. phase 86


## Load the data

In [1]:
from zooniverse.utils.data_format import data_prep
from pathlib import Path

## Input Path of all the data
input_path =Path("/Users/christian/data/zooniverse")

reprocess = False # if True, the raw classification data is reprocessed. If False, the data is loaded from disk

# Phase Selection
# phase_tag = "Iguanas 1st launch"
# phase_tag = "Iguanas 2nd launch"
phase_tag = "Iguanas 3rd launch"


debug = False # debugging with a smaller dataset
plot_diagrams = False # plot the diagrams to disk for the clustering methods
show_plots = False # show the plots in the notebook
user_threshold = None # in a number, filter records which have less than these user interactions.


### use either the subset of the subset
use_gold_standard_subset = "expert_goldstandard" # Use the X-T2-GS-results-5th-0s as the basis
# use_gold_standard_subset = "expert" # Use the expert-GS-Xphase as the basis

# Location for the analysis Results
output_path = Path(input_path.joinpath(f"2024_03_07_{use_gold_standard_subset}_analysis").joinpath(phase_tag))
output_path.mkdir(exist_ok=True, parents=True)

# Location for plots
output_plot_path = output_path.joinpath("plots")
output_plot_path.mkdir(parents=True, exist_ok=True)

if reprocess:
    ds_stats = data_prep(phase_tag=phase_tag, 
                         output_path=output_path, 
                         input_path=input_path,
                         filter_combination=use_gold_standard_subset)
    print(ds_stats)
    


In [2]:

import pandas as pd
from zooniverse.analysis import get_annotation_count_stats
from zooniverse.utils.filters import filter_df_user_threshold
from zooniverse.config import get_config

## Look into the config
This Config points to all files necessary for the analysis + the result files

In [3]:
config = get_config(phase_tag=phase_tag, input_path=input_path, output_path=output_path)
config

{'annotations_source': PosixPath('/Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv'),
 'goldstandard_data': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/expert-GS-3rdphase_renamed.csv'),
 'gold_standard_image_subset': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/3-T2-GS-results-5th-0s.csv'),
 'image_source': None,
 'yes_no_dataset': PosixPath('/Users/christian/data/zooniverse/2024_03_07_expert_goldstandard_analysis/Iguanas 3rd launch/yes_no_dataset_Iguanas 3rd launch.csv'),
 'flat_dataset': PosixPath('/Users/christian/data/zooniverse/2024_03_07_expert_goldstandard_analysis/Iguanas 3rd launch/flat_dataset_Iguanas 3rd launch.csv'),
 'merged_dataset': PosixPath('/Users/christian/data/zooniverse/2024_03_07_expert_goldstandard_analysis/Iguanas 3rd launch/merged_dataset_gold_standard_expert_Iguanas 3rd launch_filtered.csv'),
 'gold_standard_and_expert_count': PosixPath

### Optional Debugging

In [4]:
if plot_diagrams == False:
    output_plot_path = None


df_merged_dataset = pd.read_csv(config["merged_dataset"])


df_goldstandard_expert_count = pd.read_csv(config["goldstandard_data"], sep=";")

## Debugging helpers
if phase_tag == "Iguanas 1st launch":    
    if debug:

        df_merged_dataset = df_merged_dataset[df_merged_dataset.image_name.isin(["SFM01-2-2-2_333.jpg", "SFM01-2-2-2_334.jpg", "SFM01-2-2-3_201.jpg"])]

elif phase_tag == "Iguanas 2nd launch":
    if debug:
        df_merged_dataset = df_merged_dataset[
           df_merged_dataset.image_name.isin(["FMO03-1_65.jpg", "FMO03-1_72.jpg", "MBN04-2_182.jpg", "EGI08-2_78.jpg"])]
           # df_merged_dataset.image_name.isin(["FMO03-1_72.jpg"])]

    
elif phase_tag == "Iguanas 3rd launch":

    # this user is a spammer
    df_merged_dataset = df_merged_dataset[df_merged_dataset.user_id != 2581179]
    
    if debug:
        df_merged_dataset = df_merged_dataset[
           df_merged_dataset.image_name.isin(["FMO03-2_70.jpg", "MBN04-2_182.jpg", "EGI08-2_78.jpg"])]
            
    


## Look at the data


In [5]:
## Look at the data
df_merged_dataset.drop("user_name", axis=1)


,Unnamed: 0,flight_site_code,image_name,subject_id,x,y,tool_label,phase_tag,user_id
0,57,CaboIbebetsonS,PCIS01-5_67.jpg,78961972,301.422882,51.112278,"Others (females, young males, juveniles)",Iguanas 3rd launch,2494963.0
1,58,CaboIbebetsonS,PCIS01-5_67.jpg,78961972,35.903500,468.096375,"Others (females, young males, juveniles)",Iguanas 3rd launch,2494963.0
2,70,WestCoastB,GWB01-3_152.jpg,78925551,728.559448,181.467453,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN
3,71,WestCoastB,GWB01-3_152.jpg,78925551,601.206055,277.385895,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN
4,125,SouthCoastH,ESCH02-1_323.jpg,78965007,247.383331,56.599998,Adult Male not in a lek,Iguanas 3rd launch,2400702.0
...,...,...,...,...,...,...,...,...,...
7396,104634,SouthCoastH,ESCH02-1_174.jpg,78964907,688.849609,135.619003,Adult Male with a lek,Iguanas 3rd launch,2159576.0
7397,113862,GEB02,GEB02-3_197.jpg,78922625,496.962006,433.519104,Adult Male not in a lek,Iguanas 3rd launch,NaN
7398,113863,GEB02,GEB02-3_197.jpg,78922625,496.542480,416.344574,Adult Male not in a lek,Iguanas 3rd launch,NaN
7399,113866,GEB02,GEB02-3_197.jpg,78922625,502.057251,430.122284,"Others (females, young males, juveniles)",Iguanas 3rd launch,2647442.0


### Filter User if necessary and Marks


In [6]:
print(f"Before filtering: {df_merged_dataset.subject_id.nunique()}")
# There images in which some people said there are iguanas, but then didn't mark them. Clustering with fewer than 3 dots doesn't make sense
if user_threshold is not None:
    print(f"filtering records which have less than {user_threshold} interactions.")
    df_merged_dataset = filter_df_user_threshold(df_merged_dataset, user_threshold=user_threshold)
    
    
from zooniverse.utils.filters import filter_remove_marks
# Check if partials are still in the data. There shouldn't be any
df_merged_dataset = filter_remove_marks(df_merged_dataset)


2024-03-07 13:02:25.994 | WARNING  | zooniverse.utils.filters:filter_remove_marks:38 - removed 0 partial marks
2024-03-07 13:02:25.994 | WARNING  | zooniverse.utils.filters:filter_remove_marks:39 - After filter_func 87 images are left


Before filtering: 87


### Are there anonymous users in the data?
There should be

In [7]:
df_merged_dataset[df_merged_dataset.user_id.isnull().values]

,Unnamed: 0,flight_site_code,image_name,subject_id,x,y,tool_label,phase_tag,user_id,user_name
2,70,WestCoastB,GWB01-3_152.jpg,78925551,728.559448,181.467453,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,not-logged-in-1bdc6ad4144048748e1e
3,71,WestCoastB,GWB01-3_152.jpg,78925551,601.206055,277.385895,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,not-logged-in-1bdc6ad4144048748e1e
213,2466,SouthCoastH,ESCH02-1_409.jpg,78965058,579.152344,626.292969,Adult Male with a lek,Iguanas 3rd launch,NaN,not-logged-in-17a47a9cb6c229f138be
214,2467,SouthCoastH,ESCH02-1_409.jpg,78965058,371.527344,414.289062,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,not-logged-in-17a47a9cb6c229f138be
215,2468,SouthCoastH,ESCH02-1_409.jpg,78965058,377.582031,374.734375,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,not-logged-in-17a47a9cb6c229f138be
...,...,...,...,...,...,...,...,...,...,...
7361,104052,WestCoast,PWC01-1_61.jpg,78962844,613.449280,901.053284,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,not-logged-in-f2eea36ffbe6b791be19
7362,104053,WestCoast,PWC01-1_61.jpg,78962844,549.657471,899.486877,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,not-logged-in-f2eea36ffbe6b791be19
7397,113862,GEB02,GEB02-3_197.jpg,78922625,496.962006,433.519104,Adult Male not in a lek,Iguanas 3rd launch,NaN,not-logged-in-e9f1111b45a6ee28bd33
7398,113863,GEB02,GEB02-3_197.jpg,78922625,496.542480,416.344574,Adult Male not in a lek,Iguanas 3rd launch,NaN,not-logged-in-0f458e24523988619154


In [8]:
df_merged_dataset["subject_id"].nunique()

87

In [9]:
## After filtering there
df_merged_dataset

,Unnamed: 0,flight_site_code,image_name,subject_id,x,y,tool_label,phase_tag,user_id,user_name
0,57,CaboIbebetsonS,PCIS01-5_67.jpg,78961972,301.422882,51.112278,"Others (females, young males, juveniles)",Iguanas 3rd launch,2494963.0,N.Bois
1,58,CaboIbebetsonS,PCIS01-5_67.jpg,78961972,35.903500,468.096375,"Others (females, young males, juveniles)",Iguanas 3rd launch,2494963.0,N.Bois
2,70,WestCoastB,GWB01-3_152.jpg,78925551,728.559448,181.467453,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,not-logged-in-1bdc6ad4144048748e1e
3,71,WestCoastB,GWB01-3_152.jpg,78925551,601.206055,277.385895,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,not-logged-in-1bdc6ad4144048748e1e
4,125,SouthCoastH,ESCH02-1_323.jpg,78965007,247.383331,56.599998,Adult Male not in a lek,Iguanas 3rd launch,2400702.0,MoMo11
...,...,...,...,...,...,...,...,...,...,...
7396,104634,SouthCoastH,ESCH02-1_174.jpg,78964907,688.849609,135.619003,Adult Male with a lek,Iguanas 3rd launch,2159576.0,987520
7397,113862,GEB02,GEB02-3_197.jpg,78922625,496.962006,433.519104,Adult Male not in a lek,Iguanas 3rd launch,NaN,not-logged-in-e9f1111b45a6ee28bd33
7398,113863,GEB02,GEB02-3_197.jpg,78922625,496.542480,416.344574,Adult Male not in a lek,Iguanas 3rd launch,NaN,not-logged-in-0f458e24523988619154
7399,113866,GEB02,GEB02-3_197.jpg,78922625,502.057251,430.122284,"Others (females, young males, juveniles)",Iguanas 3rd launch,2647442.0,CaylinShuford


In [10]:
# how many marks per user
df_merged_dataset[["user_id", "x"]].groupby("user_id").count().head()

,x
user_id,
3427.0,13
4358.0,2
120287.0,2
156713.0,3
251333.0,8


### gold standard data
For reference

In [11]:
df_goldstandard_expert_count[df_goldstandard_expert_count["image_name"].isin(["SFM01-2-2-2_282.jpg", "SFM01-2-2-2_323.jpg"])]

,subspecies,island,site_name,subject_group,image_name,subject_id,presence_absence,count_male-lek,count_male-no-lek,count_others,count_partial,count_total,quality,condition,comment


In [12]:
df_goldstandard_expert_count.count_total.sum()

388

In [13]:
# look at the
df_goldstandard_expert_count.count()

subspecies           1156
island               1156
site_name            1156
subject_group        1156
image_name           1156
subject_id           1156
presence_absence     1156
count_male-lek       1156
count_male-no-lek    1156
count_others         1156
count_partial        1156
count_total          1156
quality               117
condition             116
comment                21
dtype: int64

In [14]:
fsum = df_goldstandard_expert_count[
    df_goldstandard_expert_count.image_name.isin(df_merged_dataset.image_name.unique())]

print(f"filtering the zooniverse classifications dataset for gold standard images the count_total of iguanas is: {fsum.count_total.sum()}, but it should be {df_goldstandard_expert_count.count_total.sum()}")
fsum


filtering the zooniverse classifications dataset for gold standard images the count_total of iguanas is: 351, but it should be 388


,subspecies,island,site_name,subject_group,image_name,subject_id,presence_absence,count_male-lek,count_male-no-lek,count_others,count_partial,count_total,quality,condition,comment
22,A. c. hayampi,Marchena,BahiaNegra,MBN1,MBN04-2_182.jpg,78926344,N,0,0,0,0,0,NaN,NaN,NaN
100,A. c. hayampi,Marchena,BahiaNegraD,MBBD1,MBBD02-2_248.jpg,78928708,Y,0,1,1,0,2,Good,NaN,NaN
208,A. c. sielmanni,Pinta,CaboIbebetson,PCI,PCI01-2_111.jpg,78938221,Y,1,0,3,0,4,Good,Visible,NaN
238,A. c. sielmanni,Pinta,CaboIbebetson,PCI,PCI02-1_92.jpg,78938603,Y,0,1,0,0,1,Good,Visible,NaN
274,A. c. sielmanni,Pinta,CaboIbebetsonC,PCIC,PCIC01-1_113.jpg,78938992,Y,0,0,2,0,2,Good,Hard,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131,A. c. nanus,Genovesa,WestCoastB,GWB,GWB01-2_300.jpg,78925388,Y,0,0,1,0,1,Good,Visible,NaN
1135,A. c. nanus,Genovesa,WestCoastB,GWB,GWB01-2_421.jpg,78925457,Y,0,0,1,0,1,Good,Visible,NaN
1136,A. c. nanus,Genovesa,WestCoastB,GWB,GWB01-2_430.jpg,78925467,Y,0,0,1,0,1,Good,Visible,NaN
1144,A. c. nanus,Genovesa,WestCoastB,GWB,GWB01-3_12.jpg,78925536,Y,0,0,1,0,1,Good,Visible,NaN


In [15]:
# How many images are left in the zooniverse dataset?
len(list(df_merged_dataset.image_name.unique()))

87

In [16]:
#Is there an image in the goldstandard, which is not in the classifcations?
len(set(df_goldstandard_expert_count.subject_id) - set(df_merged_dataset.subject_id.unique()))


df_goldstandard_expert_count.count_total.sum()
# df_merged_dataset[df_merged_dataset.image_name.isin(["SRL01-1-2_105.jpg"])]

388

In [17]:
df_goldstandard_expert_count[df_goldstandard_expert_count.image_name.isin(["SRL01-1-2_105.jpg"])]

,subspecies,island,site_name,subject_group,image_name,subject_id,presence_absence,count_male-lek,count_male-no-lek,count_others,count_partial,count_total,quality,condition,comment


In [18]:
T2_GS_results_5th_0s = pd.read_csv(config["gold_standard_image_subset"], sep=";")
T2_GS_results_5th_0s

,subject_id,Median0s,Mean0s,Max0s,Std0s,Median0s.r,Mean0s.r,Mode0s
0,78922029,2.0,1.928571,5,0.997249,2,2,2
1,78922093,9.0,11.500000,32,10.406729,9,12,9
2,78922433,1.0,1.000000,1,0.000000,1,1,1
3,78922625,1.0,1.095238,3,0.436436,1,1,1
4,78924089,2.0,1.750000,2,0.462910,2,2,2
...,...,...,...,...,...,...,...,...
82,78965032,2.0,1.964286,4,0.881167,2,2,2
83,78965058,5.0,5.620690,12,1.449478,5,6,5
84,78965066,2.0,2.272727,4,0.631085,2,2,2
85,78965103,7.0,7.464286,13,2.411118,7,7,5


## The gold standard vs. the expert count

In [19]:
# Double checking for the counts
gstd_5th = df_goldstandard_expert_count[df_goldstandard_expert_count.subject_id.isin(T2_GS_results_5th_0s.subject_id)].count_total.sum()
print(f"If the expert count ({config['goldstandard_data']})  is filtered for the subject ids in {config['gold_standard_image_subset']} the count_total is {gstd_5th} iguanas")

If the expert count (/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/expert-GS-3rdphase_renamed.csv)  is filtered for the subject ids in /Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/3-T2-GS-results-5th-0s.csv the count_total is 351 iguanas


In [20]:
len(set(T2_GS_results_5th_0s.subject_id.unique()) - set(df_goldstandard_expert_count.subject_id))
df_goldstandard_expert_count["count_total"].sum()

388

In [21]:
df_merged_dataset["subject_id"].nunique()

87

In [22]:

df_goldstandard_expert_count = df_goldstandard_expert_count[
    df_goldstandard_expert_count.subject_id.isin(df_merged_dataset.subject_id.unique())]
df_goldstandard_expert_count = df_goldstandard_expert_count[["image_name", "subject_id", "count_total"]]

df_goldstandard_expert_count["count_total"].sum()

351

In [23]:
## plot some of the marks
from zooniverse.utils.plotting import plot_zooniverse_user_marks_v2
# FMO03-1_65
# EIG05-1_83.jpg # phase 
# MBN04-2_182.jpg # phase 3
# df_merged_dataset_filtered = df_merged_dataset[df_merged_dataset.image_name.isin(["ESCG02-1_19.jpg"])]
if phase_tag in["Iguanas 1st launch", "Iguanas 2nd launch"]  and  ( plot_diagrams or show_plots ) :
    for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
        
        ## plot the marks
        markers_plot_path = plot_zooniverse_user_marks_v2(df_image_name,
                                                          image_path=df_image_name.iloc[0]["image_path"],
                                                          image_name=image_name,
                                                          output_path=output_plot_path, show=show_plots, title=f"Markers for {image_name}", fig_size=(5,5))
        

## Clustering

### Basic Statics like mean, median, mode

In [24]:
from sklearn.metrics import mean_squared_error
from zooniverse.analysis import kmeans_knee, get_mark_overview

basic_stats = []
kmeans_knee_stats = []
kmeans_silouettes = []
mse_errors = {}


for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
    annotations_count = get_mark_overview(df_image_name)


    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"])


    ### basic statistics like mean, median
    basic_stats.append(annotations_count_stats)
    

df_basic_stats = pd.DataFrame(basic_stats)    

df_comparison = df_goldstandard_expert_count.merge(df_basic_stats, on='image_name', how='left')

df_comparison["count_total"].sum()
df_goldstandard_expert_count["count_total"].sum()

351

In [25]:
# There might be records with too few annotations
df_comparison[(df_comparison.count_total > 0) & (df_comparison.sum_annotations_count < 5)].sort_values(by="users", ascending=False)

,image_name,subject_id,count_total,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count


In [26]:
# images with an expert count of more than 0 and less than 5 different users
df_comparison[(df_comparison.count_total > 0) & (df_comparison.users < 5)].sort_values(by="users", ascending=False)


,image_name,subject_id,count_total,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count


In [27]:
df_comparison["count_total"].sum()

351

### Fill NaN values with 0 because the errors can't be calculated otherwise

In [28]:

## Fill NaN values with 0 because the errors can't be calculated otherwise
df_comparison.fillna(0, inplace=True)


In [29]:

mse_errors["median_count_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.median_count,
                                                     squared=False)
mse_errors["mean_count_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.mean_count, squared=False)
mse_errors["mode_count_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.mode_count, squared=False)

pd.Series(mse_errors)

median_count_rmse    1.555487
mean_count_rmse      1.521801
mode_count_rmse      2.031363
dtype: float64

It can be seen the knee method has a very high Root mean squared error. 

In [30]:
df_comparison

,image_name,subject_id,count_total,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count
0,MBN04-2_182.jpg,78926344,0,2.0,1.71,1,7,12,"[1, 1, 1, 2, 2, 2, 3]"
1,MBBD02-2_248.jpg,78928708,2,2.0,2.12,2,17,36,"[1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, ..."
2,PCI01-2_111.jpg,78938221,4,3.0,3.50,3,20,70,"[2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, ..."
3,PCI02-1_92.jpg,78938603,1,1.0,1.21,1,19,23,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,PCIC01-1_113.jpg,78938992,2,2.0,1.75,2,8,14,"[1, 1, 1, 2, 2, 2, 2, 3]"
...,...,...,...,...,...,...,...,...,...
82,GWB01-2_300.jpg,78925388,1,1.0,1.00,1,9,9,"[1, 1, 1, 1, 1, 1, 1, 1, 1]"
83,GWB01-2_421.jpg,78925457,1,1.0,1.11,1,9,10,"[1, 1, 1, 1, 1, 1, 1, 1, 2]"
84,GWB01-2_430.jpg,78925467,1,1.0,1.11,1,9,10,"[1, 1, 1, 1, 1, 1, 1, 1, 2]"
85,GWB01-3_12.jpg,78925536,1,1.0,1.04,1,26,27,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [31]:

pd.Series(mse_errors)

median_count_rmse    1.555487
mean_count_rmse      1.521801
mode_count_rmse      2.031363
dtype: float64

In [32]:
df_comparison

,image_name,subject_id,count_total,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count
0,MBN04-2_182.jpg,78926344,0,2.0,1.71,1,7,12,"[1, 1, 1, 2, 2, 2, 3]"
1,MBBD02-2_248.jpg,78928708,2,2.0,2.12,2,17,36,"[1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, ..."
2,PCI01-2_111.jpg,78938221,4,3.0,3.50,3,20,70,"[2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, ..."
3,PCI02-1_92.jpg,78938603,1,1.0,1.21,1,19,23,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,PCIC01-1_113.jpg,78938992,2,2.0,1.75,2,8,14,"[1, 1, 1, 2, 2, 2, 2, 3]"
...,...,...,...,...,...,...,...,...,...
82,GWB01-2_300.jpg,78925388,1,1.0,1.00,1,9,9,"[1, 1, 1, 1, 1, 1, 1, 1, 1]"
83,GWB01-2_421.jpg,78925457,1,1.0,1.11,1,9,10,"[1, 1, 1, 1, 1, 1, 1, 1, 2]"
84,GWB01-2_430.jpg,78925467,1,1.0,1.11,1,9,10,"[1, 1, 1, 1, 1, 1, 1, 1, 2]"
85,GWB01-3_12.jpg,78925536,1,1.0,1.04,1,26,27,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [33]:
df_comparison.sum()

image_name               MBN04-2_182.jpgMBBD02-2_248.jpgPCI01-2_111.jpg...
subject_id                                                      6869056455
count_total                                                            351
median_count                                                         315.0
mean_count                                                          318.73
mode_count                                                             304
users                                                                 1606
sum_annotations_count                                                 6733
annotations_count        [1, 1, 1, 2, 2, 2, 3, 1, 1, 2, 2, 2, 2, 2, 2, ...
dtype: object

### DBSCAN clustering and take the variant with the best silouette score for each image


In [34]:
### The old variant
# from zooniverse.analysis import compare_dbscan_hyp_v2
# 
# eps_variants = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
# min_samples_variants = [3, 5, 8, 10]
# if debug:
#     eps_variants = [0.3]
#     min_samples_variants = [3]
# params = [(eps, min_samples) for eps in eps_variants for min_samples in min_samples_variants]
# 
# db_scan_results = {}
# db_scan_best_results = []
# db_scan_best_bic_results = []
# for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
# 
#     dbscan_localization = compare_dbscan_hyp_v2(
#         # phase_tag=phase_tag,
#         params=params,
#         df_flat=df_image_name,
#         # output_path=output_path,
#         output_plot_path=output_plot_path,
#         plot=show_plots,
#         
#     )
# 
#     db_scan_results[image_name] = pd.DataFrame(dbscan_localization)
#     
#     # TODO Here lies the main problem with DBSCAN.
#     ## DBSCAN tends to classfy all points as noise if min_samples is too high. Often only a single user marked an iguana.
#     ## Sillouette Scoring needs a minimum of 2 clusters
#     ## if there are points in decent radius they will belong to a cluster
#     # if pd.DataFrame(dbscan_localization).dbscan_count.max() == 1:
#     #     db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
#     #     db_scan_best_bic_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
#     # # If two or more cluster seem to exists take ones with the best BIC or Silouette score
#     # else:  
#     # take the best result by silouette score if there are more clusters then 1
#     ## TODO make the sorting deterministic
#     db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_silouette_score", ascending=False).iloc[0])
#     
# df_dbscan_localization = pd.concat([*db_scan_results.values()])
# df_scan_best_results = pd.DataFrame(db_scan_best_results)



In [35]:
# df_scan_best_results

In [36]:
## fixes the problem with the silouette score sorting
from zooniverse.analysis import compare_dbscan_hyp_v2

eps_variants = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
min_samples_variants = [3, 5, 8, 10]
if debug:
    eps_variants = [0.3]
    min_samples_variants = [3]
params = [(eps, min_samples) for eps in eps_variants for min_samples in min_samples_variants]

db_scan_results = {}
db_scan_best_results = []
db_scan_best_bic_results = []
for image_name, df_image_name in df_merged_dataset.groupby("image_name"):

    dbscan_localization = compare_dbscan_hyp_v2(
        # phase_tag=phase_tag,
        params=params,
        df_flat=df_image_name,
        # output_path=output_path,
        output_plot_path=output_plot_path,
        plot=show_plots,
        
    )

    db_scan_results[image_name] = pd.DataFrame(dbscan_localization)
    
    # TODO Here lies the main problem with DBSCAN.
    # DBSCAN tends to classfy all points as noise if min_samples is too high. Often only a single user marked an iguana.
    # Sillouette Scoring needs a minimum of 2 clusters
    # if there are points in decent radius they will belong to a cluster
    if pd.DataFrame(dbscan_localization).dbscan_count.max() == 1:
        db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
        db_scan_best_bic_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
        # If two or more cluster seem to exists take ones with the best Silouette score
    else:  
        # take the best result by silouette score if there are more clusters then 1
        db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_silouette_score", ascending=False).iloc[0])
    
df_dbscan_localization = pd.concat([*db_scan_results.values()])
df_scan_best_results = pd.DataFrame(db_scan_best_results)



2024-03-07 13:02:27.118 | WARNING  | zooniverse.utils.basic_clustering:bic_score:126 - only one point in the cluster
/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:127: RuntimeWarning: invalid value encountered in scalar divide
  variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
2024-03-07 13:02:27.156 | WARNING  | zooniverse.utils.basic_clustering:bic_score:126 - only one point in the cluster
/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:127: RuntimeWarning: invalid value encountered in scalar divide
  variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
2024-03-07 13:02:27.284 | WARNING  | zooniverse.utils.basic_clustering:bic_score:126 - only one point in the cluster
/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:127: RuntimeWarning: invalid value encountered in scalar divide
  variance = 

In [37]:
df_scan_best_results

,dbscan_count,dbscan_noise,dbscan_silouette_score,dbscan_BIC_score,image_name,eps,min_samples
10,18,28,0.616352,-4182.334018,ESCH01-1_13.jpg,0.10,8
4,9,12,0.716935,-1112.370289,ESCH01-1_19.jpg,0.05,3
20,8,1,0.861200,NaN,ESCH01-1_21.jpg,0.40,3
27,4,1,0.931680,NaN,ESCH01-1_22.jpg,0.50,10
16,2,1,0.874491,NaN,ESCH01-1_23.jpg,0.30,3
...,...,...,...,...,...,...,...
27,4,3,0.664809,-3621.622512,PWC01-1_61.jpg,0.50,10
12,7,1,0.806874,NaN,PWC01-1_62.jpg,0.20,3
24,4,3,0.752832,-293.623536,PWC01-4_231.jpg,0.50,3
4,2,8,0.158432,-182.195215,PWC01-5_34.jpg,0.05,3


Here it can be seen why the silouette score is difficult because it is often undefined.

In [38]:
## save the combinations of parameters, which maximized the silouette score.

df_dbscan_localization.to_csv(config["dbscan_hyperparam_grid"])
df_scan_best_results

,dbscan_count,dbscan_noise,dbscan_silouette_score,dbscan_BIC_score,image_name,eps,min_samples
10,18,28,0.616352,-4182.334018,ESCH01-1_13.jpg,0.10,8
4,9,12,0.716935,-1112.370289,ESCH01-1_19.jpg,0.05,3
20,8,1,0.861200,NaN,ESCH01-1_21.jpg,0.40,3
27,4,1,0.931680,NaN,ESCH01-1_22.jpg,0.50,10
16,2,1,0.874491,NaN,ESCH01-1_23.jpg,0.30,3
...,...,...,...,...,...,...,...
27,4,3,0.664809,-3621.622512,PWC01-1_61.jpg,0.50,10
12,7,1,0.806874,NaN,PWC01-1_62.jpg,0.20,3
24,4,3,0.752832,-293.623536,PWC01-4_231.jpg,0.50,3
4,2,8,0.158432,-182.195215,PWC01-5_34.jpg,0.05,3


In [39]:
df_scan_best_results.rename(columns={"dbscan_count": "dbscan_count_sil" }, inplace=True)

df_comparison = df_comparison.merge(df_scan_best_results, on='image_name', how='left')

In [40]:
df_comparison.fillna(0, inplace=True)

mse_errors["dbscan_count_sil_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.dbscan_count_sil, squared=False)

pd.Series(mse_errors)

median_count_rmse        1.555487
mean_count_rmse          1.521801
mode_count_rmse          2.031363
dbscan_count_sil_rmse    2.138858
dtype: float64

In [41]:

df_comparison = df_comparison.drop(["dbscan_noise", "dbscan_silouette_score", "eps", "min_samples", "dbscan_BIC_score", "with_noise", "bic_avg"], axis=1, errors="ignore")
df_comparison

,image_name,subject_id,count_total,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count,dbscan_count_sil
0,MBN04-2_182.jpg,78926344,0,2.0,1.71,1,7,12,"[1, 1, 1, 2, 2, 2, 3]",2
1,MBBD02-2_248.jpg,78928708,2,2.0,2.12,2,17,36,"[1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, ...",2
2,PCI01-2_111.jpg,78938221,4,3.0,3.50,3,20,70,"[2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, ...",5
3,PCI02-1_92.jpg,78938603,1,1.0,1.21,1,19,23,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
4,PCIC01-1_113.jpg,78938992,2,2.0,1.75,2,8,14,"[1, 1, 1, 2, 2, 2, 2, 3]",2
...,...,...,...,...,...,...,...,...,...,...
82,GWB01-2_300.jpg,78925388,1,1.0,1.00,1,9,9,"[1, 1, 1, 1, 1, 1, 1, 1, 1]",0
83,GWB01-2_421.jpg,78925457,1,1.0,1.11,1,9,10,"[1, 1, 1, 1, 1, 1, 1, 1, 2]",1
84,GWB01-2_430.jpg,78925467,1,1.0,1.11,1,9,10,"[1, 1, 1, 1, 1, 1, 1, 1, 2]",1
85,GWB01-3_12.jpg,78925536,1,1.0,1.04,1,26,27,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2


### HDBSCAN clustering for each image

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html#sklearn.cluster.HDBSCAN states: "A distance threshold. Clusters below this value will be merged."



In [42]:
from zooniverse.analysis import HDBSCAN_Wrapper

hdbscan_values = []

eps_variants = [0.0] # 0 is the default
min_cluster_sizes = [5] # 5 is the default


for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
    annotations_count = get_mark_overview(df_image_name)
    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"])
    
    
    if df_image_name.shape[0] >= 5: # if less then min_cluster_sizes points are available clustering makes no sense
        params = [(eps, min_cluster_size, max_cluster_size) 
                    for eps in eps_variants
                    for min_cluster_size in min_cluster_sizes
                    for max_cluster_size in [None]
              ]

        df_hdbscan = HDBSCAN_Wrapper(df_marks=df_image_name[["x", "y"]],
                                     annotations_count=annotations_count,
                                     output_path=output_plot_path,
                                     plot=show_plots,
                                     show=show_plots,
                                     image_name=image_name,
                                     params=params)
        hdbscan_values.append(df_hdbscan)


df_hdbscan = pd.concat(hdbscan_values)



2024-03-07 13:02:44.297 | WARNING  | zooniverse.utils.basic_clustering:bic_score:126 - only one point in the cluster
/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:127: RuntimeWarning: invalid value encountered in scalar divide
  variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
2024-03-07 13:02:44.346 | WARNING  | zooniverse.utils.basic_clustering:bic_score:126 - only one point in the cluster
/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:127: RuntimeWarning: invalid value encountered in scalar divide
  variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
2024-03-07 13:02:44.358 | WARNING  | zooniverse.utils.basic_clustering:bic_score:126 - only one point in the cluster
/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:127: RuntimeWarning: invalid value encountered in scalar divide
  variance = 

In [43]:
df_hdbscan.drop(["with_noise", "bic_avg"], axis=1, inplace=True)
df_hdbscan

,image_name,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,ESCH01-1_13.jpg,19,0.0,5,None,26
0,ESCH01-1_19.jpg,9,0.0,5,None,8
0,ESCH01-1_21.jpg,10,0.0,5,None,5
0,ESCH01-1_22.jpg,5,0.0,5,None,6
0,ESCH01-1_23.jpg,2,0.0,5,None,0
...,...,...,...,...,...,...
0,PWC01-1_61.jpg,14,0.0,5,None,36
0,PWC01-1_62.jpg,11,0.0,5,None,5
0,PWC01-4_231.jpg,3,0.0,5,None,0
0,PWC01-5_34.jpg,1,0.0,5,None,12


In [44]:
df_comparison = df_comparison.merge(df_hdbscan, on='image_name', how='left')


df_comparison["count_total"].sum()

351

In [45]:
df_comparison.to_csv(config["comparison_dataset"])
print(f"saved {config['comparison_dataset']}")

saved /Users/christian/data/zooniverse/2024_03_07_expert_goldstandard_analysis/Iguanas 3rd launch/Iguanas 3rd launch_method_comparison.csv


In [46]:
df_comparison.count_total.fillna(0, inplace=True)
df_comparison.HDBSCAN_count.fillna(0, inplace=True)

mse_errors["hdbscan_count_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.HDBSCAN_count, squared=False)


# A look into the results
Root Means Squared Error for the different methods

In [47]:
df_rmse = pd.DataFrame(pd.Series(mse_errors).sort_values())

df_rmse.to_csv(config["rmse_errors"])
df_rmse

,0
hdbscan_count_rmse,1.174440
mean_count_rmse,1.521801
median_count_rmse,1.555487
mode_count_rmse,2.031363
dbscan_count_sil_rmse,2.138858


## The sum of the clustering
What is the sum of the methods

In [48]:

df_comparison_sum = df_comparison[["count_total", "median_count", "mean_count", "mode_count", "dbscan_count_sil", "HDBSCAN_count"]].sum().sort_values()
df_comparison_sum.to_csv(config["method_sums"])
df_comparison_sum

mode_count          304.00
median_count        315.00
mean_count          318.73
dbscan_count_sil    321.00
count_total         351.00
HDBSCAN_count       357.00
dtype: float64

In [49]:
print(f"phase_tag: {phase_tag}, user_threshold: {user_threshold}")

phase_tag: Iguanas 3rd launch, user_threshold: None


## Compare the numbers
The counts are only for images which were in the dataset after filtering.

In [50]:
if phase_tag == "Iguanas 1st launch" and not debug:
    if user_threshold == 3:
        assert df_comparison_sum["mode_count"] == 215
        assert df_comparison_sum["dbscan_count_sil"] == 222
        assert df_comparison_sum["median_count"] == 228.5
        assert df_comparison_sum["HDBSCAN_count"] == 244
        assert df_comparison_sum["count_total"] == 323
    if user_threshold is None:
        assert df_comparison_sum["mode_count"] == 221
        assert df_comparison_sum["dbscan_count_sil"] == 224
        assert df_comparison_sum["median_count"] == 235.5
        assert df_comparison_sum["HDBSCAN_count"] == 247
        assert df_comparison_sum["count_total"] == 331
        
if phase_tag == "Iguanas 2nd launch" and not debug:
    if user_threshold == 3:
        assert df_comparison_sum["mode_count"] == 502
        assert df_comparison_sum["dbscan_count_sil"] == 484
        assert df_comparison_sum["median_count"] == 475
        assert df_comparison_sum["HDBSCAN_count"] == 541
        assert df_comparison_sum["count_total"] == 586
    if user_threshold is None:
        assert df_comparison_sum["mode_count"] == 511
        assert df_comparison_sum["dbscan_count_sil"] == 484
        assert df_comparison_sum["median_count"] == 484.5
        assert df_comparison_sum["HDBSCAN_count"] == 541.0
        assert df_comparison_sum["count_total"] == 589.0
        
if phase_tag == "Iguanas 3rd launch" and not debug:
    if user_threshold == 3:
        assert df_comparison_sum["mode_count"] == 302
        assert df_comparison_sum["dbscan_count_sil"] == 309
        assert df_comparison_sum["median_count"] == 313
        assert df_comparison_sum["HDBSCAN_count"] == 357
        assert df_comparison_sum["count_total"] == 351
    if user_threshold is None:
        assert df_comparison_sum["mode_count"] == 304
        assert df_comparison_sum["dbscan_count_sil"] == 309
        assert df_comparison_sum["median_count"] == 315
        assert df_comparison_sum["HDBSCAN_count"] == 357
        assert df_comparison_sum["count_total"] == 351


AssertionError: 

### Sum of all the Methods

In [51]:
print(f"{config['method_sums'].name}")
pd.read_csv(config["method_sums"])

Iguanas 3rd launch_method_sums.csv


,Unnamed: 0,0
0,mode_count,304.00
1,median_count,315.00
2,mean_count,318.73
3,dbscan_count_sil,321.00
4,count_total,351.00
5,HDBSCAN_count,357.00


### Root Mean Squared Error

In [52]:
print(f"{config['rmse_errors'].name}")
pd.read_csv(config["rmse_errors"])

Iguanas 3rd launch_rmse_errors.csv


,Unnamed: 0,0
0,hdbscan_count_rmse,1.174440
1,mean_count_rmse,1.521801
2,median_count_rmse,1.555487
3,mode_count_rmse,2.031363
4,dbscan_count_sil_rmse,2.138858


### Comparison per Image Level

In [53]:
print(f"load {config['comparison_dataset']}")
pd.read_csv(config["comparison_dataset"])

load /Users/christian/data/zooniverse/2024_03_07_expert_goldstandard_analysis/Iguanas 3rd launch/Iguanas 3rd launch_method_comparison.csv


,Unnamed: 0,image_name,subject_id,count_total,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,0,MBN04-2_182.jpg,78926344,0,2.0,1.71,1,7,12,"[1, 1, 1, 2, 2, 2, 3]",2,2.0,0.0,5.0,NaN,0.0
1,1,MBBD02-2_248.jpg,78928708,2,2.0,2.12,2,17,36,"[1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, ...",2,2.0,0.0,5.0,NaN,0.0
2,2,PCI01-2_111.jpg,78938221,4,3.0,3.50,3,20,70,"[2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, ...",5,4.0,0.0,5.0,NaN,6.0
3,3,PCI02-1_92.jpg,78938603,1,1.0,1.21,1,19,23,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,1.0,0.0,5.0,NaN,12.0
4,4,PCIC01-1_113.jpg,78938992,2,2.0,1.75,2,8,14,"[1, 1, 1, 2, 2, 2, 2, 3]",2,2.0,0.0,5.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,82,GWB01-2_300.jpg,78925388,1,1.0,1.00,1,9,9,"[1, 1, 1, 1, 1, 1, 1, 1, 1]",0,1.0,0.0,5.0,NaN,3.0
83,83,GWB01-2_421.jpg,78925457,1,1.0,1.11,1,9,10,"[1, 1, 1, 1, 1, 1, 1, 1, 2]",1,1.0,0.0,5.0,NaN,5.0
84,84,GWB01-2_430.jpg,78925467,1,1.0,1.11,1,9,10,"[1, 1, 1, 1, 1, 1, 1, 1, 2]",1,1.0,0.0,5.0,NaN,5.0
85,85,GWB01-3_12.jpg,78925536,1,1.0,1.04,1,26,27,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,1.0,0.0,5.0,NaN,14.0


## Discussion:
Clustering works, it yields better numbers than just taking mode,median or mean annotations from the volunteers, because it takes the spatial location of the marker dots into consideration.

